In [2]:
%matplotlib inline
import random
import torch

In [4]:
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [5]:
features


tensor([[ 0.4800, -0.5458],
        [-0.2173,  0.3229],
        [ 1.1786,  0.3650],
        ...,
        [ 0.8490,  0.0331],
        [-0.0378, -1.0917],
        [ 0.5931,  0.2381]])

In [6]:
labels

tensor([[  6.9974],
        [  2.6643],
        [  5.3202],
        [ 10.4746],
        [  4.3283],
        [ 11.6752],
        [  7.5350],
        [  3.4997],
        [ -1.2930],
        [  4.6363],
        [  6.4003],
        [  2.1129],
        [  1.8244],
        [  3.8602],
        [  4.8866],
        [  5.3784],
        [  7.4922],
        [  6.2122],
        [  0.7620],
        [  7.5248],
        [  7.9775],
        [ -5.5303],
        [  5.2356],
        [ -0.8251],
        [  0.0765],
        [  2.4810],
        [  3.6074],
        [  2.3474],
        [  6.1652],
        [  6.0962],
        [  3.0775],
        [  3.8717],
        [  5.4308],
        [  0.3516],
        [  7.3299],
        [  8.1655],
        [  2.8600],
        [  3.3230],
        [  0.1948],
        [  5.8133],
        [  4.5020],
        [  2.2072],
        [  2.2383],
        [  1.8807],
        [  6.7311],
        [ -0.2809],
        [  1.2389],
        [  0.8892],
        [  6.1512],
        [  7.2067],


In [7]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

In [8]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[-0.9674, -2.5416],
        [-0.3901,  1.2503],
        [ 1.3921, -1.2513],
        [-2.1671, -1.7554],
        [-0.7855, -0.3180],
        [-1.0078, -0.1054],
        [-0.3904,  0.5863],
        [-0.0839,  0.0892],
        [ 0.6120,  0.9185],
        [-0.8260, -1.2800]]) 
 tensor([[10.9051],
        [-0.8016],
        [11.2420],
        [ 5.8248],
        [ 3.7010],
        [ 2.5231],
        [ 1.4186],
        [ 3.7336],
        [ 2.2957],
        [ 6.9051]])


In [9]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [10]:
def linreg(X, w, b):  #@save
    """线性回归模型"""
    return torch.matmul(X, w) + b

In [11]:
def squared_loss(y_hat, y):  #@save
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [12]:
def sgd(params, lr, batch_size):  #@save
    """小批量随机梯度下降"""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [13]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # X和y的小批量损失
        # 因为l形状是(batch_size,1)，而不是一个标量。l中的所有元素被加到一起，
        # 并以此计算关于[w,b]的梯度
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # 使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.035916
epoch 2, loss 0.000125
epoch 3, loss 0.000051


In [14]:
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

w的估计误差: tensor([ 0.0001, -0.0002], grad_fn=<SubBackward0>)
b的估计误差: tensor([0.0007], grad_fn=<RsubBackward1>)
